In [1]:
#592 Machine Learning Code

In [1]:
# imports
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from nltk.stem import PorterStemmer
from num2words import num2words
from nltk.corpus import words
from nltk.corpus import wordnet
from nltk.corpus import words as nltk_words
nltk.download('words')
from collections import defaultdict

[nltk_data] Downloading package words to /Users/Nelson/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# Load in Data

with open("Tesla_Comments_p.json", "r") as file:
    tesla_data_p = json.load(file)
with open("Tesla_Comments_c.json", "r") as file:
    tesla_data_c = json.load(file)
    
with open("GE_Comments_p.json", "r") as file:
    ge_data_p = json.load(file)
with open("GE_Comments_c.json", "r") as file:
    ge_data_c = json.load(file)
     
with open("AMD_Comments_p.json", "r") as file:
    amd_data_p = json.load(file)
with open("AMD_Comments_c.json", "r") as file:
    amd_data_c = json.load(file)
    
with open("NVDA_Comments_p.json", "r") as file:
    nvda_data_p = json.load(file)
with open("NVDA_Comments_c.json", "r") as file:
    nvda_data_c = json.load(file)

    
tesla_stock_data = pd.read_csv("TSLA.csv")
ge_stock_data = pd.read_csv("GE.csv")
amd_stock_data = pd.read_csv("AMD.csv")
nvda_stock_data = pd.read_csv("NVDA.csv")

posts = [tesla_data_p, ge_data_p, amd_data_p, nvda_data_p]
comms = [tesla_data_c, ge_data_c, amd_data_c, nvda_data_c]
stocs = [tesla_stock_data, ge_stock_data, amd_stock_data, nvda_stock_data]
tickers = ["TSLA","GE","AMD","NVDA"]

In [3]:
# Combining and Cleaning Finance and Social Media data

combined = pd.DataFrame()
for comm, post, stoc, ticker in zip(comms, posts, stocs, tickers):
    
    # create columns
    df_comments = pd.json_normalize(comm).transpose()
    df_posts = pd.json_normalize(post).transpose()
    #df_posts.rename(columns={0,"Posts"})
    df_posts.columns = ['Posts']
    
    df_stock_info = stoc.set_index("Date")
    
    # join columns
    joined_df = df_stock_info.join(df_comments)
    joined_df = joined_df.join(df_posts)

    # add price column chance
    joined_df["Price_Change"] = joined_df["Close"] - joined_df["Open"]

    def dire(x):
        if x>0:
            return "up"
        elif x<0:
            return "down"
        else:
            return "no change"

    joined_df["Direction"] = joined_df["Price_Change"].apply(dire) 

    joined_df["Ticker"] = ticker


    joined_df2 = joined_df.rename(columns={"Open":"Open_Price","High":"High_Price","Low":"Low_Price","Close":"Close_Price",
                              "Volume":"Trade_Volume","Direction":"Price_Direction",0:"Comments"})
    
    joined_df3 = joined_df2
    
    combined = pd.concat([combined,joined_df3])
    
# drop rows where posts are empty so comments are nan
combined = combined.dropna()

In [4]:
# Sentiment Creation

Post_Sentiment = []
Comment_Sentiment = []

sia = SentimentIntensityAnalyzer()

Total_Positive_Sentiment_Comm = []
Total_Negative_Sentiment_Comm = []
Total_Positive_Sentiment_Post = []
Total_Negative_Sentiment_Post = []
Number_of_Comments = []

for p, c in zip(combined["Posts"], combined["Comments"]):
    
    # Comment Sentiment
    tot_pos = 0
    tot_neg = 0

    for comment in c:
        pos = sia.polarity_scores(comment)["pos"]
        tot_pos = tot_pos + pos
        
        neg = sia.polarity_scores(comment)["neg"]
        tot_neg = tot_neg + neg
        
    if tot_pos > tot_neg:
        Comment_Sentiment.append("Pos")
    else:
        Comment_Sentiment.append("Neg")
        
    Total_Positive_Sentiment_Comm.append(tot_pos)
    Total_Negative_Sentiment_Comm.append(tot_neg)
       
        
    # Post Sentiment
    tot_pos = 0
    tot_neg = 0

    for post in p:
        pos = sia.polarity_scores(post)["pos"]
        tot_pos = tot_pos + pos
        
        neg = sia.polarity_scores(post)["neg"]
        tot_neg = tot_neg + neg

    if tot_pos > tot_neg:
        Post_Sentiment.append("Pos")
    else:
        Post_Sentiment.append("Neg")
    
    Total_Positive_Sentiment_Post.append(tot_pos)
    Total_Negative_Sentiment_Post.append(tot_neg) 
    
    Number_of_Comments.append(len(c))
    
    
combined["Post_Sentiment"] = Post_Sentiment
combined["Comment_Sentiment"] = Comment_Sentiment
combined["Total_Positive_Sentiment_Comm"] = Total_Positive_Sentiment_Comm
combined["Total_Negative_Sentiment_Comm"] = Total_Negative_Sentiment_Comm
combined["Total_Positive_Sentiment_Post"] = Total_Positive_Sentiment_Post
combined["Total_Negative_Sentiment_Post"] =Total_Negative_Sentiment_Post
combined["Number_of_Comments"] = Number_of_Comments

combined2 = combined[["Ticker","Posts","Post_Sentiment","Total_Positive_Sentiment_Post",
                      "Total_Negative_Sentiment_Post", "Comments", "Comment_Sentiment",
                      "Total_Positive_Sentiment_Comm", 
                      "Total_Negative_Sentiment_Comm","Number_of_Comments","Price_Change",
                      "Price_Direction","Open_Price","Close_Price","High_Price",
                      "Low_Price","Trade_Volume"]]

# Remove rows with empty posts.
mask = []
for val in combined2["Posts"]:
    if val[0] == "":
        mask.append(False)
    else:
        mask.append(True)
        
comb3 = combined2[mask]

In [37]:
#comb3[:10]

In [6]:

# Randomize our dataframe for Machine Learning below
comb3 = comb3.sample(frac=1, random_state=1)


In [7]:
variables = pd.DataFrame()

variables['Post_Sentiment'] = np.where(comb3['Post_Sentiment']=="Pos", 1, 0)
variables['Comment_Sentiment'] = np.where(comb3['Comment_Sentiment']=="Pos", 1, 0)



variables["Numb_Comm_ized"] = (np.array(comb3["Number_of_Comments"]) - 
                               np.mean(np.array(comb3["Number_of_Comments"]))
                              ) / np.std(np.array(comb3["Number_of_Comments"]))


variables["Numb_Comm_ized"] = (np.array(comb3["Number_of_Comments"]) - 
                               np.mean(np.array(comb3["Number_of_Comments"]))
                              ) / np.std(np.array(comb3["Number_of_Comments"]))

# this feature will be about amount of change, and not direction
comb3['Price_Change'] = np.where(comb3['Price_Change']>=0, comb3['Price_Change'], abs(comb3['Price_Change']))
variables["Abs_Val_Price_Change"] = (np.array(comb3["Price_Change"]) - 
                               np.mean(np.array(comb3["Price_Change"]))
                              ) / np.std(np.array(comb3["Price_Change"]))

variables["Open_Price"] = (np.array(comb3["Open_Price"]) - 
                               np.mean(np.array(comb3["Open_Price"]))
                              ) / np.std(np.array(comb3["Open_Price"]))

variables["Close_Price"] = (np.array(comb3["Close_Price"]) - 
                               np.mean(np.array(comb3["Close_Price"]))
                              ) / np.std(np.array(comb3["Close_Price"]))

variables["High_Price"] = (np.array(comb3["High_Price"]) - 
                               np.mean(np.array(comb3["High_Price"]))
                              ) / np.std(np.array(comb3["High_Price"]))

variables["Low_Price"] = (np.array(comb3["Low_Price"]) - 
                               np.mean(np.array(comb3["Low_Price"]))
                              ) / np.std(np.array(comb3["Low_Price"]))

variables["Trade_Volume"] = (np.array(comb3["Trade_Volume"]) - 
                               np.mean(np.array(comb3["Trade_Volume"]))
                              ) / np.std(np.array(comb3["Trade_Volume"]))

y_var = pd.DataFrame()
y_var['Price_Direction'] = np.where(comb3['Price_Direction']=="up", 1, 0)

In [36]:
#variables

In [35]:
#y_var

In [10]:
# Create Machine Learning Classification structure.  Use Support Vector Machines

   # say 10 x features, based on sentiment Post, sentiment Comment, price..... 
       # .... predict Y price direction or price change

tot_a = 0
for i in range(50):    
    
    X_train, X_test, y_train, y_test = train_test_split(variables, y_var, test_size=0.25)

    # SVM Classifier
    clf = svm.SVC()  # no specification parameter so defaults to non-linear separation, clustery.

    # Training 
    clf.fit(X_train, y_train)

    # Make predictions based on correlations from training.
    y_pred = clf.predict(X_test)

    a = metrics.accuracy_score(y_test, y_pred)
    tot_a += a
    #print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.p

In [11]:
print('Average Accuracy', tot_a / 50)

Average Accuracy 0.546823529411765


In [12]:
# Test with only the sentiment features, no numbers.

vs = variables[["Post_Sentiment", "Comment_Sentiment"]]

total_a = 0
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(vs, y_var, test_size=0.25)


    # SVM Classifier
    clf = svm.SVC()  # no specification parameter so defaults to non-linear separation, clustery.

    # Training 
    clf.fit(X_train, y_train)

    # Make predictions based on correlations from training.
    y_pred = clf.predict(X_test)

    a = metrics.accuracy_score(y_test, y_pred)
    total_a += a
    #print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

In [13]:
print('Average Accuracy', total_a / 50)

Average Accuracy 0.5414117647058828


In [14]:
# Test with-OUT the sentiment features, ONLY numbers.

vs_B = variables[['Numb_Comm_ized',
       'Abs_Val_Price_Change', 'Open_Price', 'Close_Price', 'High_Price',
       'Low_Price', 'Trade_Volume']]

t = 0
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(vs_B, y_var, test_size=0.25)


    # SVM Classifier
    clf = svm.SVC()  # no specification parameter so defaults to non-linear separation, clustery.

    # Training 
    clf.fit(X_train, y_train)

    # Make predictions based on correlations from training.
    y_pred = clf.predict(X_test)

    a = metrics.accuracy_score(y_test, y_pred)
    t += a
    #print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

In [15]:
print('Average Accuracy', t / 50)

Average Accuracy 0.5430588235294121


In [16]:



# The average accuracy from the above analysis is about 54 percent in all three..
    # .. tests above.  Showing sentiment and the financial numbers do the same.
    
    


In [17]:
# Lets test our SVM classifier on other data.

test = []
for val in list(y_var.iterrows())[:100]:
    if val[1][0] == 0:
        test.append(0)
    else:
        test.append(1)

# only small portion of the trainin is wrong.
for val in list(y_var.iterrows())[100:150]:
    if val[1][0] == 0:
        test.append(1)
    else:
        test.append(0)
        
for val in list(y_var.iterrows())[150:]:
    if val[1][0] == 0:
        test.append(0)
    else:
        test.append(1)

test_df = pd.DataFrame(test)

t = 0
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(test_df, y_var, test_size=0.25)


    # SVM Classifier
    clf = svm.SVC()  # no specification parameter so defaults to non-linear separation, clustery.

    # Training 
    clf.fit(X_train, y_train)

    # Make predictions based on correlations from training.
    y_pred = clf.predict(X_test)

    a = metrics.accuracy_score(y_test, y_pred)
    t += a
    #print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

In [18]:
print('Average Accuracy', t / 50)

Average Accuracy 0.855764705882353


In [19]:
#  Our classifier seems appropriate, getting 85% accuracy when that is expected.

In [20]:




### Adding TF_IDF as features





In [21]:

# Step 1: Look at total posts and comments of a single record (one day) as a document

new_records = []
for p, c in zip(comb3["Posts"], comb3["Comments"]):
    p2 = p[0]
    c.append(p2)
    new_records.append(c)

In [22]:
    
# Step 2: Do all preprocessing and append our unique list of terms across all documents

stop_words = stopwords.words('english')
stemmer = PorterStemmer()

dd = defaultdict(lambda: defaultdict(int))
words_per_doc = dict()
for doc_n, record in enumerate(new_records): # remember each record will be viewed as a document in our TF_IDF
    total_words_in_rec = 0 
    for string in record:
        for word in string.split():
            word_lower = word.lower()            
            
            # check if word is a stop word, if it's likely a word, and remove piece of punctuation
            if (word_lower not in stop_words) and (word_lower[:-1].isalpha()): 
                if word_lower[-1].isalpha():
                    dd[word_lower][doc_n] = dd[word_lower][doc_n] + 1
                    total_words_in_rec += 1 
                else:
                    dd[word_lower[:-1]][doc_n] = dd[word_lower[:-1]][doc_n] + 1
                    total_words_in_rec += 1
                    
    words_per_doc[doc_n] = total_words_in_rec
                    

In [34]:
#dd

In [124]:
# 339 Documents
len(words_per_doc)

339

In [125]:
# 22,625 Unique Words
len(dd.keys())

22625

In [23]:
# Step 3 Calculate Scores to create the final TF_IDF (output in a dictionary)


TF_IDF = defaultdict(lambda: defaultdict(int))  # if score of 0, not included in the structure
for term, docs in dd.items():
    
    num_docs_per_term = len(docs)
    
    for doc, freq in docs.items():
        
        #tf = how many times term appears in document.  so freq / total terms in doc
        TF = freq / words_per_doc[doc]
    
        # idf = log(number of docs in corpus / how many docs have this term) + 1
        IDF = np.log(339 / num_docs_per_term) + 1
    
        # score(t,d) = tf * idf 
        score = TF * IDF
    
        TF_IDF[term][doc] = score


In [33]:
#TF_IDF

In [24]:

# Code to Fill in 0's for a full TD_IDF
TF_IDF_B = defaultdict(lambda: defaultdict(int))

for term, docs in TF_IDF.items():
    
    existing_docs = defaultdict(int)
    for doc, score in docs.items():
        existing_docs[doc] += score
    
    for i in range(339):
        if i in existing_docs:
            TF_IDF_B[term][i] = existing_docs[i]
        else:
            TF_IDF_B[term][i] = 0



In [32]:
#TF_IDF_B

In [25]:
## Next, Create our X Features for Machine Learning by extracting the Document Vector from the TF_IDF

# we should have 22,625 features, one for each unique word in corpus
  # (sample 1) doc0 [score of t1, score of t2,...  may have lot of 0's in it]
    
feature_dic = defaultdict(list)
terms = []
for term, docs in TF_IDF_B.items():
    terms.append(term)
    # for loop below must run 22,625 times for each do
    for doc, score in docs.items():
        
        # we can run this line if wanted to see terms in order with the score
        #feature_dic[doc].append((term,score))
        
        feature_dic[doc].append(score)

In [31]:
#feature_dic

In [28]:
NL = [] 
for doc, vector in feature_dic.items():
    NL.append(vector)

In [30]:
features = pd.DataFrame(NL, columns=terms)
#features

In [156]:
# Now, lets test SVM with only our new features.

t = 0
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(features, y_var, test_size=0.25)


    # SVM Classifier
    clf = svm.SVC()  # no specification parameter so defaults to non-linear separation, cluster-like.

    # Training 
    clf.fit(X_train, y_train)

    # Make predictions based on correlations from training.
    y_pred = clf.predict(X_test)

    a = metrics.accuracy_score(y_test, y_pred)
    t += a
        
print('Average Accuracy', t / 5)

/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nelson/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma w

Average Accuracy 0.548235294117647
